In [ ]:
# 1º Passo – Importar as bibliotecas que serão utilizadas

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import argparse
import pandas as pd
import cv2
import numpy as np
import os
import seaborn as sn
%matplotlib inline

In [ ]:
# 2º Passo – Carregar os arrays construídos na etapa referente ao pré-processamento de dados e normalizar os dados do input

X_train = np.load("/content/drive/My Drive/Python/COVID/Arrays/X_Train.npy")
X_train = X_train/255

Y_train = np.load("/content/drive/My Drive/Python/COVID/Arrays/Y_Train.npy")

In [ ]:
# 3º Passo – Dividir os dados em dados de treinamento e dados de teste

X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train, test_size = 0.2, random_state = 40)

print(f"X_train shape: {X_train.shape} Y_train shape {Y_train.shape}")
print(f"X_test shape: {X_test.shape} Y_test shape {Y_test.shape}")

In [ ]:
# 4º Passo – Determinando a arquitetura do modelo que será treinado

bModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(237, 237, 3)))

tModel = bModel.output
tModel = AveragePooling2D(pool_size=(4, 4))(tModel)
tModel = Flatten(name="flatten")(tModel)
tModel = Dense(64, activation="relu")(tModel)
tModel = Dropout(0.2)(tModel)
tModel = Dense(1, activation="sigmoid")(tModel)

model = Model(inputs=bModel.input, outputs=tModel)

In [ ]:
# 5º Passo – Determinar os hyperparameters e compilar o modelo

INIT_LR = 1e-3
EPOCHS = 50
BS = 8

for layer in bModel.layers:
	layer.trainable = False

opt = Adam(lr=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# 6º Passo – Treinar o modelo

X = model.fit(X_train, Y_train, batch_size=BS,validation_split=0.1, epochs=EPOCHS)

model.save("/content/drive/My Drive/Python/COVID/model/modeloc_1.hdf5")

In [ ]:
# 7º Passo – Observar a acurácia do modelo e a função de perda

plt.plot(X.history['accuracy'])
plt.plot(X.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(X.history['loss'])
plt.plot(X.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

model.evaluate(X_test,Y_test)

In [ ]:
# 8º Passo – Observar quais imagens o modelo acertou

plt.figure(figsize=(20,20))
i = 0
for i,image in enumerate(X_test):
    plt.subplot(7,9,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    img = np.expand_dims(X_test[i],axis = 0)
    x_pred = model.predict(img)[0]
    x0 = x_pred[0]
    
    if x0 > 0.5:
      label = "COVID"
    else:
      label = "NORMAL"
    
    if Y_test[i] == 1:
      label_test = "COVID"
    else:
      label_test = "NORMAL"
    plt.xlabel(f"Label Predict = {label} \n Label Correct = {label_test}")
    i += 1
plt.savefig('/content/drive/My Drive/Python/COVID/model/modelo_1.pdf')


In [ ]:
# 9º Passo – Construir uma matriz de confusão

ypredict = model.predict(X_test)

ypredictc = []

for value in ypredict:
  x0 = value [0]
  # x1 = value [1]
  if x0 > 0.5:
    ypredictc.append(1)

  else:
    ypredictc.append(0)

resultado = np.array(ypredictc)



x = confusion_matrix(y_true=Y_test,y_pred=resultado)
x = x/X_test.shape[0]

y = pd.DataFrame(x,index = ["NORMAL","COVID"],columns=["NORMAL","COVID"])
plt.figure(figsize = (10,7))

fig = sn.heatmap(y, annot=True,cmap="Greens").get_figure()
fig.savefig("plot.jpg") 